In [6]:
# Этот код парсит сайт https://openphish.com и собирает информацию о фишинговых ссылках. 
# В процессе работы программа будет выполняться в течение 1 часа и пауза между запросами будет равна 300 секундам.

# Функция parse_openphish() использует библиотеку requests для отправки GET-запросов на сайт. 
# При каждом запросе будет получен html-код сайта, который парсится с помощью библиотеки BeautifulSoup.

# Из html-кода программа ищет таблицу, а затем перебирает все строки в таблице, 
# извлекает данные из ячеек таблицы и сохраняет их в двух словарях: 
# unique_urls (хранит уникальные фишинговые ссылки) и brands (хранит бренды, которые атаковали фишингом, 
# и количество атак на каждый бренд).

# Функция возвращает время начала и окончания сбора данных, количество уникальных URL и топ-3 марки, которые были атакованы.
# Импортируем необходимые библиотеки: requests, csv, time и BeautifulSoup из bs4
import requests
import csv
import time
from bs4 import BeautifulSoup

# Определяем функцию parse_openphish с параметрами по умолчанию interval=300, hour=1.
def parse_openphish(interval=300, hour=1):
    # Задаем начальное время скрипта
    start_time = time.time()
    # Задаем конечное время скрипта, равное начальному времени + количество часов, умноженных на 3600 секунд
    end_time = start_time + hour * 3600
    # URL сайта openphish
    url = 'https://openphish.com/'
    # Множество unique_urls для хранения уникальных URL
    unique_urls = set()
    # Словарь brands для хранения статистики по брендам
    brands = {}
    # Задаем заголовок User-Agent
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    # Бесконечный цикл, пока время не станет больше end_time
    while time.time() < end_time:
        # Отправляем GET-запрос к URL с заданными заголовками
        response = requests.get(url, headers=headers)
        # Создаем объект BeautifulSoup из полученных данных и парсера html.parser.Парсинг содержимого сайта с помощью BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        # Поиск таблицы с данными в HTML-документе
        table = soup.find('table')
        #Перебор строк таблицы (начиная со второй, т.к. первая строка содержит заголовок)
        for row in table.find_all('tr')[1:]:
            # Поиск ячеек в строке
            cells = row.find_all('td')
            # Извлечение URL фишинговой страницы и бренда из ячеек
            phishing_url = cells[0].text.strip()
            brand = cells[1].text.strip()
            # Добавление URL в множество уникальных URL
            unique_urls.add(phishing_url)
            # Обновление счетчика для данного бренда
            if brand in brands:
                brands[brand] += 1
            else:
                brands[brand] = 1\
        # Задержка 
        time.sleep(interval)
# Вычисляем ТОП-3 марки, участвующие в фишинговых атаках
# Сортировка словаря brands по значению, указывается через параметр key, 
# который является функцией lambda, которая выбирает значение элемента (x[1]).
# Обрезаем список до первых трех элементов, используя оператор [:3].
    top_brands = sorted(brands.items(), key=lambda x: x[1], reverse=True)[:3]

# Возвращаем результаты работы функции
    return start_time, end_time, len(unique_urls), top_brands
if __name__ == '__main__':
    # Вызываем функцию parse_openphish
    start, end, num_urls, top_brands = parse_openphish()
    
    # Форматируем время начала и конца работы функции в читаемый формат
    start_time_formatted = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(start))
    end_time_formatted = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(end))
    
    # Выводим результаты работы функции
    print('Start time:', start_time_formatted)
    print('End time:', end_time_formatted)
    print('Number of unique URLs:', num_urls)
    print('Top 3 attacked brands:', top_brands)


Start time: 2023-02-02 05:40:02
End time: 2023-02-02 06:40:02
Number of unique URLs: 61
Top 3 attacked brands: [('Australian Government', 12), ('Apple Inc.', 12), ('Generic/Spear Phishing', 12)]
